In [1]:
import dgl
from dgl.data import DGLDataset
import torch
import os
import pandas as pd
import numpy as np
import torch
import itertools
import numpy as np
import scipy.sparse as sp
import dgl.nn as dglnn
import torch.nn as nn
import torch.nn.functional as F
import dgl.function as fn
from dgl.nn.pytorch import HGTConv
from startup_data_set import *
from PredictorClasses import *
from CustomMetrics import *
from CustomUtilities import *

In [4]:
graph = COMP4222Dataset_hetero()
g = graph[0]

In [5]:
print(g)

Graph(num_nodes={'investor': 7594, 'startup': 17852},
      num_edges={('investor', 'i_s', 'startup'): 45621},
      metagraph=[('investor', 'startup', 'i_s')])


In [103]:
neg_g = construct_negative_hetero_graph(g,1,("investor","raise","startup"))

In [104]:
investor_feats = g.nodes['investor'].data['feat']
startup_feats = g.nodes['startup'].data['feat']
node_features = {'investor': investor_feats, 
                 'startup': startup_feats}

In [105]:
train_g, test_g, valid_g = generate_train_test_valid_hetero_graph(g)


In [106]:
class Model(nn.Module):
    def __init__(self, 
                 in_features_size,
                 head_size,
                 num_heads,
                 num_ntypes,
                 num_etypes,
                 dropout=0.2,
                 use_norm=False):
        super().__init__()
        
        self.conv = HGTConv(in_features_size, head_size, num_heads, num_ntypes,num_etypes,dropout,use_norm)
        self.pred = HeteroDotProductPredictor()
    def forward(self, g, neg_g, x, ntype, etype):
        h = self.conv(g, x, ntype, etype)
        return self.pred(g, h, etype), self.pred(neg_g, h, etype)

In [107]:
class HeteroDotProductPredictor(nn.Module):
    def forward(self, graph, h, etype):
        # h contains the node representations for each node type computed from
        # the GNN defined in the previous section (Section 5.1).
        with graph.local_scope():
            graph.ndata['h'] = h
            graph.apply_edges(fn.u_dot_v('h', 'h', 'score'), etype=etype)
            return graph.edges[etype].data['score']

In [111]:
def compute_loss(pos_score, neg_score):
    # Margin loss
    n_edges = pos_score.shape[0]
    return (1 - pos_score + neg_score.view(n_edges, -1)).clamp(min=0).mean()

print(train_g)
t_g = dgl.to_homogeneous(train_g)
k = 5
num_etypes = int(len(train_g.ntypes))
num_ntypes = int(len(train_g.etypes))
# in_features_size = train_g.nodes["investor"].data["feat"].shape[1]
print(t_g.ndata)
num_heads = 5
head_size = in_features_size//num_heads

model = Model(in_features_size, 
              head_size, 
              num_heads, 
              int(len(train_g.ntypes)),
              int(len(train_g.etypes)))

investor_feats = g.nodes['investor'].data['feat']
startup_feats = g.nodes['startup'].data['feat']
node_features = {'investor': investor_feats, 'startup': startup_feats}


etype = torch.tensor([i % num_etypes for i in range(train_g.num_edges())])
ntype = torch.tensor([i % num_ntypes for i in range(train_g.num_nodes())])

print(etype.shape)
print(ntype.shape)
opt = torch.optim.Adam(model.parameters())
for epoch in range(10):
    negative_graph = construct_negative_hetero_graph(train_g, k, ("investor","raise","startup"))
    
    pos_score, neg_score = model(t_g, negative_graph, train_g.nodes["investor"].data["feat"],etype,ntype )
    
    loss = compute_loss(pos_score, neg_score)
    opt.zero_grad()
    loss.backward()
    opt.step()
    print(loss.item())

Graph(num_nodes={'investor': 7594, 'startup': 21483},
      num_edges={('investor', 'raise', 'startup'): 48787},
      metagraph=[('investor', 'startup', 'raise')])
{'_ID': tensor([    0,     1,     2,  ..., 21480, 21481, 21482]), '_TYPE': tensor([0, 0, 0,  ..., 1, 1, 1])}
torch.Size([48787])
torch.Size([29077])


IndexError: index out of range in self